# Introduction

**Sentinel-1 Normalised Radar Backscatter**  
Sentinel-1 Normalised Radar Backscatter (S1-NRB) is a newly developed Analysis Ready Data (ARD) product of the European Space Agency that offers high-quality, radiometrically terrain corrected (RTC) Synthetic Aperture Radar (SAR) backscatter. Each product includes annotation image datasets (e.g., local incident angle) and metadata for the product as well as source datasets.   
You can find more detailed information about the S1-NRB product [here](https://sentinel.esa.int/web/sentinel/sentinel-1-ard-normalised-radar-backscatter-nrb-product). 

**SpatioTemporal Asset Catalog (STAC)**  
All S1-NRB products include metadata in JSON format compliant with the [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) specification.   
STAC uses several sub-specifications ([Item](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md), [Collection](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md) & [Catalog](https://github.com/radiantearth/stac-spec/blob/master/catalog-spec/catalog-spec.md)) to create a hierarchical structure that enables efficient querying and access of large volumes of geospatial data.

**This example notebook will give a short demonstration of how the STAC metadata provided with S1-NRB products can be utilized with little effort to efficiently analyze and visualize large datasets.**

# Getting started

After following the [installation instructions](https://s1-nrb.readthedocs.io/en/latest/general/installation.html) you need to install a few additional packages into the activated conda environment to reproduce all steps presented in the following example notebook.

```bash
conda activate nrb_env   
conda install jupyterlab stackstac rioxarray xarray_leaflet
```

Instead of importing all packages now, they will successively be imported throughout the notebook:

In [2]:
import numpy as np
import stackstac
from S1_NRB.metadata.stac import make_catalog

Let's assume you have a collection of S1-NRB scenes located somewhere on your local disk or a fileserver. As mentioned in the [Introduction](#Introduction), each S1-NRB scene includes metadata as a STAC Item, describing the scene's temporal, spatial and product specific properties.

The **only step necessary to get going** and start analysing your collection of scenes, is the creation of STAC Collection and Catalog files, which connect individual STAC Items. For this purpose, `S1_NRB` includes the utility function [make_catalog](https://s1-nrb.readthedocs.io/en/latest/api.html#S1_NRB.metadata.stac.make_catalog), which will create these files for you. Please note that `make_catalog` expects a directory structure based on MGRS tile IDs, which allows for efficient data querying and access. It will take care of reorganizing your S1-NRB scenes if this directory structure doesn't exist yet.

In [3]:
nrb_catalog = make_catalog(directory='./NRB_thuringia', silent=True)

./NRB_thuringia
and the NRB products it contains will be reorganized into subdirectories based on unique MGRS tile IDs if this directory structure does not yet exist. 
Do you wish to continue? [yes|no]  yes



#### New STAC endpoint created: ./NRB_thuringia/catalog.json


   
The STAC Catalog can then be used with libraries such as [stackstac](https://github.com/gjoseph92/stackstac), which _"turns a STAC Collection into a lazy xarray.DataArray, backed by dask"._ 

The term _lazy_ describes a [method of execution](https://tutorial.dask.org/01x_lazy.html) that only computes results when actually needed and thereby enables computations on larger-than-memory datasets. _[xarray](https://xarray.pydata.org/en/stable/index.html)_ is a Python library for working with labeled multi-dimensional arrays of data, while the Python library _[dask](https://docs.dask.org/en/latest/)_ facilitates parallel computing in a flexible way.

Compatibility with [odc-stac](https://github.com/opendatacube/odc-stac), a very [similar library](https://github.com/opendatacube/odc-stac/issues/54) to stackstac, will be tested in the near future.

In [4]:
aoi = (10.638066, 50.708415, 11.686751, 50.975775)
ds = stackstac.stack(items=nrb_catalog, bounds_latlon=aoi, 
                     dtype=np.dtype('float32'), chunksize=(-1, 1, 1024, 1024))
ds

,Array,Chunk
Bytes,107.39 GiB,484.00 MiB
Shape,"(121, 10, 3189, 7471)","(121, 1, 1024, 1024)"
Count,340 Tasks,320 Chunks
Type,float32,numpy.ndarray


As you can see in the output above, the collection of S1-NRB scenes was successfully loaded as an `xarray.DataArray`. The metadata attributes included in all STAC Items are now available as coordinate arrays (see [here](https://docs.xarray.dev/en/stable/user-guide/terminology.html#term-Coordinate) for clarification of Xarray's terminology) and can be utilized during analysis. This will be explored in the next chapter.  

# Data Exploration

## Spatial

In [5]:
import xarray_leaflet
from ipyleaflet import Map
from matplotlib import pyplot as plt
import rioxarray

# To get an idea about the spatial extent of the data, we select the gamma0 backscatter data in VH polarization for 
# a single time step and compute the raster values into memory
vh = ds.sel(band='vh', time='2020-01-04').squeeze().compute()

# For visualization with xarray_leaflet, reprojection to the coordinate reference system WGS84 is necessary 
# and we limit the value range so that outliers don't affect the color scale 
vh_4326 = vh.rio.reproject("EPSG:4326", nodata=np.nan)
vh_4326_q99 = vh_4326.where((vh_4326 > vh_4326.quantile(0.01)) & 
                            (vh_4326 < vh_4326.quantile(0.99)), drop=True) 

# Visualize the array on a Leaflet map 
m = Map(center=[50.9, 11.2], zoom=11)
l = vh_4326_q99.leaflet.plot(m, colormap=plt.cm.gray)
m

Url()

Map(center=[50.9, 11.2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

## Temporal

## Bands / Attributes

# Example Analysis